# Home task for Data Scientist (GenAI) Candidate

In [1]:
#loading documents and appending to a list
from langchain_community.document_loaders import PyPDFLoader

documents = []

file_paths = ["./Tallink-Grupp-Sustainability-Report-2023.pdf", "./EE-aastaaruanne-2023-ENG.pdf"]

for path in file_paths:
    loader = PyPDFLoader(path)
    document = loader.load()
    documents.append(document)

#assigning each document to a variable
tallink = documents[0]
eesti_energy = documents[1]

In [2]:
print(len(documents),len(tallink),len(eesti_energy)) #checking upload ok

2 76 85


In [3]:
print(f"{tallink[1].page_content[:200]}\n") #looking at some of the document content (second page and first 200 characters)
print(tallink[1].metadata)

TABLE OF CONTENT
2023 Key highlights 
About this report
Welcome message from the CEO
Tallink Grupp at a glance
Group structure
Our fleet and routes
Passengers in 2023
What we do
Our business philosoph

{'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Macintosh)', 'creationdate': '2024-05-05T18:23:58+03:00', 'moddate': '2024-05-06T10:13:13+03:00', 'trapped': '/False', 'source': './Tallink-Grupp-Sustainability-Report-2023.pdf', 'total_pages': 76, 'page': 1, 'page_label': '2'}


In [4]:
print(f"{eesti_energy[3].page_content[:100]}\n") #looking at other document

Eesti Energia Unaudited Annual Report 2023
4
Contents
Interest in storage solutions skyrocketed
In s



In [5]:
#splitting documents using recursive character text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

splits = []

for doc in documents:
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True)
    doc_splits = text_splitter.split_documents(doc)
    splits.append(doc_splits)

tallink_splits = splits[0]
eesti_energy_splits = splits[1]

In [6]:
print(len(splits), len(tallink_splits), len(eesti_energy_splits))

2 190 379


In [7]:
from langchain.vectorstores import FAISS
#from langchain_core.vectorstores import InMemoryVectorStore
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize embedding model
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Creating vector stores for both documents (based on split files) with embedding model
tallink_vector_store = FAISS.from_documents(tallink_splits, embedding)
eesti_vector_store = FAISS.from_documents(eesti_energy_splits, embedding)

C:\Users\debbi\AppData\Local\Temp\ipykernel_13948\36454013.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\debbi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
tallink_vector_store

In [9]:
eesti_vector_store

In [10]:
#using similarity search to find similar content to the query from the document
results = tallink_vector_store.similarity_search(
    "What’s the company’s purpose and main activities?"
)

print(results[0])

page_content='22
I  General
Business strategy
The Group’s vision is to be the market pioneer 
in Europe by offering excellence in leisure and 
business travel and sea transportation services.
The Group’s long-term strategy is to:
• Strive for the highest level of customer satis-
faction
• Improve profitability and strengthen the 
leading position in our home markets
• Develop a wide range of quality services for 
different customers and pursue new growth 
opportunities
• Based on the demand on the core routes, 
manage optimal capacity by operating or 
chartering out vessels
• Ensure cost efficient operations
• Maintain an optimal debt level that allows 
paying sustainable dividends
A modern fleet, a wide route network, a strong 
market share and brand awareness together 
with high safety, security and environmental 
standards are the main competitive advantages 
for the Group.
These are the cornerstones for successful and 
profitable operations.' metadata={'producer': 'Adobe PDF Librar

In [11]:
results = eesti_vector_store.similarity_search(
    "What’s the company’s purpose and main activities?"
)

print(results[0])

page_content='manage Eesti Energia in accordance with the owner’s expecta-
tions, the Group’s strategy, vision and values, and applicable laws 
and regulations. We have adopted key performance indicators 
for our strategic goals, which we use to set and measure the 
achievement of goals and to evaluate the effectiveness of the 
work done. These allow us to assess whether we are on track to 
meet our goals. The Group’s strategic goals are set for a peri-
od of five years and updated annually. To achieve the strategic 
goals, managers engage and empower the staff in alignment 
with our values and Group-wide governance principles. We use 
internal communication channels to keep employees informed 
about the organisation’s goals and how we are achieving them. 
We make sure that our people have a safe work environment and 
a high work ethic. We pay our employees a competitive salary 
and we recognise and reward them. The Group’s management 
board and supervisory board are accountable to the

In [12]:
results = tallink_vector_store.similarity_search(
    "When did Tallink Hotel Riga reopen?"
)

print(results[0])

page_content='of its 4 hotels. Tallink Spa & Conference Hotel 
and Tallink Express Hotel reopened in autumn 
2020, Tallink City Hotel reopened after exten -
sive repairs in June 2021. Tallink Hotel Riga re -
opened for business in April 2023. The hotels 
welcome an average of over 450 000 guests per 
annum.
Since 2023 all Tallink Hotels are Green Key cer-
tified.
Tallink Hotels operate a number of popular res-
taurants within the hotels, such as restaurants 
Tempo, Nero and Tallink City Grill House. The 
restaurants draw crowds from local city work -
ers and dwellers as well as from among hotel 
guests, with the Sunday brunches particularly 
popular among the families. In addition to the 
hotel restaurants, Tallink Hotels also operate 
two stand-alone restaurants – Ristorante Fla -
vore and NOKNOK, both of which have already 
developed a loyal customer base during their 
many years of operation.
I  General' metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3

Together API key: b87147925336e042811802b051ba6dca4c0059cad12b262758e4347b963f0c59

In [13]:
import getpass
import os

if "TOGETHER_API_KEY" not in os.environ:
    os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter your Together API key: ")

Below option to use Langsmith API key which I have not used in this case

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

In [14]:
import pandas as pd

Below code is testing a for loop to be used with the LLM model, but instead of model response I have used 'test' purely for testing purposes

In [15]:
# testing loop to be used with model but with 'test' response instead of calling model
final_result_test = []
companies = ["Tallink", "Eesti Energy"]

questions = ["Please can you provide a summary of the document content?",
             "What’s the company’s purpose and main activities?", 
             "What’s the company’s primary focus?", 
             "What’s the company’s commitment for Risk Management?",
             "What’s the company’s commitment for Energy Management?",
             "What’s the company’s commitment for Data Security?"]

for company in companies:
    for question in questions:
        response = "test"

        row = [company, question, response]
        final_result_test.append(row)

result_df_test = pd.DataFrame(final_result_test,columns=['Company','Question','Answer'])
result_df_test

,Company,Question,Answer
0,Tallink,Please can you provide a summary of the docume...,test
1,Tallink,What’s the company’s purpose and main activities?,test
2,Tallink,What’s the company’s primary focus?,test
3,Tallink,What’s the company’s commitment for Risk Manag...,test
4,Tallink,What’s the company’s commitment for Energy Man...,test
5,Tallink,What’s the company’s commitment for Data Secur...,test
6,Eesti Energy,Please can you provide a summary of the docume...,test
7,Eesti Energy,What’s the company’s purpose and main activities?,test
8,Eesti Energy,What’s the company’s primary focus?,test
9,Eesti Energy,What’s the company’s commitment for Risk Manag...,test


Defining application steps and LLM below

In [16]:
from langchain_together import ChatTogether
from langchain import hub
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

# initialising an empty list to store final results and defining companies and questions
final_result = []
companies = ["Tallink", "Eesti Energy"]

questions = ["Please can you provide a summary of the document content?",
             "What’s the company’s purpose and main activities?", 
             "What’s the company’s primary focus?", 
             "What’s the company’s commitment for Risk Management?",
             "What’s the company’s commitment for Energy Management?",
             "What’s the company’s commitment for Data Security?"]

#Defining llm and prompt
llm = ChatTogether(
    model="meta-llama/Llama-3-70b-chat-hf",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

prompt = hub.pull("rlm/rag-prompt")

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps - different vector stores to be used depending on company
def retrieve(state: State):
    if company == "Tallink":
        retrieved_docs = tallink_vector_store.similarity_search(state["question"])
    elif company == "Eesti Energy":
        retrieved_docs = eesti_vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

c:\Users\debbi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Using a for loop to generate answers to all questions for both companies, then appending the results to a DataFrame

In [17]:
# Using a for loop to generate answers to all questions for both companies, then appending the results to a DataFrame
for company in companies:
    for question in questions:
        response = graph.invoke({"question": question})

        row = [company, question, response["answer"]]

        final_result.append(row)

result_df = pd.DataFrame(final_result,columns=['Company','Question','Answer'])
result_df

,Company,Question,Answer
0,Tallink,Please can you provide a summary of the docume...,The document appears to be an annual report or...
1,Tallink,What’s the company’s purpose and main activities?,The company's purpose is to be the market pion...
2,Tallink,What’s the company’s primary focus?,The company's primary focus is to offer excell...
3,Tallink,What’s the company’s commitment for Risk Manag...,The company's commitment to Risk Management is...
4,Tallink,What’s the company’s commitment for Energy Man...,Tallink Grupp has committed to increasing the ...
5,Tallink,What’s the company’s commitment for Data Secur...,The company is committed to safeguarding the c...
6,Eesti Energy,Please can you provide a summary of the docume...,The document appears to be the Eesti Energia U...
7,Eesti Energy,What’s the company’s purpose and main activities?,The company's purpose is to operate profitably...
8,Eesti Energy,What’s the company’s primary focus?,The company's primary focus is on leading the ...
9,Eesti Energy,What’s the company’s commitment for Risk Manag...,The company's commitment to Risk Management is...


In [18]:
# Convert DataFrame to JSON and save to a file
result_df.to_json('taskresults.json', orient='records', lines=True)

In [19]:
import json
import streamlit as st

with open('taskresults.json') as f:
    data = [json.loads(line) for line in f]

st.json(data)

# Display the JSON in the Streamlit UI
st.title("Results Viewer")
st.json(data)

2025-06-16 01:27:14.205 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 01:27:14.607 
  command:

    streamlit run C:\Users\debbi\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-06-16 01:27:14.609 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 01:27:14.609 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 01:27:14.609 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 01:27:14.609 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 01:27:14.609 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()